#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2017


# Homework 3:  Classification Cook-off: Naive Bayes vs Rocchio (plus a little bit of recommenders)

### 100 points [5% of your final grade]

### Due: Wednesday, March 29 by 11:59pm

*Goals of this homework:* Hands-on practice building and evaluating classifiers.

*Submission Instructions:* To submit your homework, rename this notebook as YOUR_UIN_hw3.ipynb. Submit this notebook via eCampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Saturday April 1 at 11:59pm.

# Part 0: Yelp review data

In this assignment, given a Yelp review, your task is to implement two classifiers to predict if the business category of this review is "food-relevant" or not, **only based on the review text**. The data is from the [Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge).

## Build the training data

First, you will need to download this data file as your training data: [training_data.json](https://drive.google.com/open?id=0B_13wIEAmbQMdzBVTndwenoxQlk) 

The training data file includes 40,000 Yelp reviews. Each line is a json-encoded review, and **you should only focus on the "text" field**. As same as in homework 1, you should tokenize the review text by using the regular expression "\W+" (we discussed it in [this Piazza post](https://piazza.com/class/ixkk1fy863r1vs?cid=29). Do NOT remove stop words. **Do casefolding but no stemming**.

The label (class) information of each review is in the "label" field. It is **either "Food-relevant" or "Food-irrelevant"**.

## Testing data

We provide 100 yelp reviews here: [testing_data.json](https://drive.google.com/open?id=0B_13wIEAmbQMbXdyTkhrZDN4Wms). The testing data file has the same format as the training data file. Again, you can get the label informaiton in the "label" field. Only use it when you evalute your classifiers.

# Part 1: Naive Bayes classifier [35 points]

In this part, you will implement a Naive Bayes classifier, which outputs the probabilities that a given review belongs to each class.

Use a mixture model that mixes the probability from the document with the general collection frequency of the word. **You should use lambda = 0.7**. Be careful about the decimal rounding since multiplying many probabilities can generate a tiny value. We will not grade on the exact probability value, so feel free to change to logorithm summation (it's not required, though). If the tie case happens, **always go to the "Food-irrelevant" side**.

### What to report

* For the entire testing dataset, report the overall accuracy.
* For the class "Food-relevant", report the precision and recall.
* For the class "Food-irrelevant", report the precision and recall.

We will also grade on the quality of your code. So make sure that your code is clear and readable.

In [38]:
# Build the naive bayes classifier
# Insert as many cells as you want
import json
import math
import re
import sys
from collections import defaultdict

lamda = 0.7
klassesCount = defaultdict(lambda: 0.0)
totalDocs = 0
vocab = set()
klassTotalWordCount = defaultdict(lambda: 0.0)
klassWordCount = defaultdict(lambda: 0.0)
klasses = ["Food-relevant", "Food-irrelevant"]

def add_review(review, klass):
    global klassesCount, totalDocs, vocab, klassTotalWordCount, klassWordCount
    totalDocs += 1
    klassesCount[klass] += 1
    temp_list = re.split('\W+', review)
    for word in temp_list:
        word = word.lower()
        vocab.add(word)
        klassTotalWordCount[klass] += 1
        klassWordCount[(klass, word)] += 1

def parse_json():
    with open('training_data.json') as data_file:
        for line in data_file:
            data = json.loads(line)
            klass = data["label"]
            review = data["text"]
            add_review(review, klass)

parse_json()
print "vocab size : ", len(vocab)
            
def nb_classify(review):
    klass_prob = {}
    temp_list = re.split('\W+', review)
    for klass in klasses:
        klass_prob[klass] = math.log(((0.0 + klassesCount[klass]) / totalDocs), 10)
        for word in temp_list:
            if word not in vocab:
                continue
            num2 = num1 = klassWordCount[(klass, word)]
            den2 = den1 = klassTotalWordCount[klass]
            if klass == klasses[0]:
                num2 += classWordCount[(klasses[1], word)]
                den2 += classTotalWordCount[klasses[1]]
            else:
                num2 += classWordCount[(klasses[0], word)]
                den2 += classTotalWordCount[klasses[0]]
            klass_prob[klass] += math.log((lamda*((0.0 + num1) / den1) + (1 - lamda)*((0.0 + num2) / den2)), 10)
    return klass_prob

 vocab size :  53138


In [39]:
# Apply your classifier on the test data. Report the results.
# Insert as many cells as you want
def test():
    confusion_matrix = defaultdict(lambda: defaultdict(lambda: 0.0))
    with open('testing_data.json') as data_file:
        total_docs = 0.0
        correct_docs = 0.0
        incorrect_docs = {}
        for line in data_file:
            total_docs += 1
            data = json.loads(line)
            klass_prob = nb_classify(data["text"])
            predicted_class = None
            max_prob = -sys.maxint -1
            for klass in klasses:
                temp_prob = klass_prob[klass]
                if temp_prob > max_prob:
                    max_prob = temp_prob
                    predicted_class = klass
            print klass_prob[classes[0]], " ", klass_prob[classes[1]], " ", predicted_class, " ", data["label"]
            if predicted_class == data["label"]:
                correct_docs += 1
                confusion_matrix[predicted_class]["True"] += 1
            else:
                incorrect_docs[total_docs] = (data["text"], data["label"])
                confusion_matrix[predicted_class]["False"] += 1
    print "accuracy : ", correct_docs / total_docs
    print klasses[0], " : "
    print "Precision : ", confusion_matrix[klasses[0]]["True"] / (confusion_matrix[klasses[0]]["True"] + confusion_matrix[klasses[0]]["False"])
    print "Recall : ", confusion_matrix[klasses[0]]["True"] / (confusion_matrix[klasses[0]]["True"] + confusion_matrix[klasses[1]]["False"])
    
    print klasses[1], " : "
    print "Precision : ", confusion_matrix[klasses[1]]["True"] / (confusion_matrix[klasses[1]]["True"] + confusion_matrix[klasses[1]]["False"])
    print "Recall : ", confusion_matrix[klasses[1]]["True"] / (confusion_matrix[klasses[1]]["True"] + confusion_matrix[klasses[0]]["False"])
    '''
    for key in incorrect_docs:
        print key, " ", incorrect_docs[key]
        print
    '''
test()


-85.9605398301   -84.6262793803   Food-relevant   Food-relevant
-44.4595026398   -41.5561409261   Food-relevant   Food-relevant
-336.043129789   -325.379015812   Food-relevant   Food-relevant
-19.9468663671   -19.1756078399   Food-relevant   Food-relevant
-143.89214258   -138.680106306   Food-relevant   Food-relevant
-159.981486473   -161.390028251   Food-irrelevant   Food-relevant
-238.641632016   -225.304594928   Food-relevant   Food-relevant
-281.695925909   -277.628365275   Food-relevant   Food-relevant
-50.9041378683   -48.5960726241   Food-relevant   Food-relevant
-20.3582462189   -19.8082169512   Food-relevant   Food-relevant
-108.001180662   -106.180150799   Food-relevant   Food-relevant
-229.617521629   -227.751351272   Food-relevant   Food-relevant
-54.302039354   -52.3646814337   Food-relevant   Food-relevant
-125.107836799   -120.939639341   Food-relevant   Food-relevant
-65.1094196263   -60.4150487226   Food-relevant   Food-relevant
-355.802774413   -348.047155238   Food-r

# Part 2: Rocchio classifier [35 points]

In this part, your job is to implement a Rocchio classifier for "food-relevant vs. food-irrelevant". You need to aggregate all the reviews of each class, and find the center. **Use the normalized raw term frequency**.


### What to report

* For the entire testing dataset, report the overall accuracy.
* For the class "Food-relevant", report the precision and recall.
* For the class "Food-irrelevant", report the precision and recall.

We will also grade on the quality of your code. So make sure that your code is clear and readable.

In [40]:
def magnitude(u):
    return math.sqrt(sum(u[i]*u[i] for i in range(len(u))))

def magnitude_map(u):
    return math.sqrt(sum(u[i]*u[i] for i in u))

def normalize(u):
    mag = magnitude(u)
    if mag == 0:
        return u
    return [float(u[i])/mag for i in range(len(u))]

def normalize_map(u):
    mag = magnitude_map(u)
    if mag == 0:
        return u
    for i in u:
        u[i] = float(u[i])/mag
    return u

def centroid(u, num_docs):
    return [float(u[i])/num_docs for i in range(len(u))]

In [53]:
# Build the Rocchio classifier
# Insert as many cells as you want
klasses = ["Food-relevant", "Food-irrelevant"]
klass_vec = defaultdict(lambda: defaultdict(lambda: 0.0))
doc_count = defaultdict(lambda: 0.0)
vocab = set()

def tokenize(review):
    word_list = re.split('\W+', review)
    temp_vec = defaultdict(lambda: 0.0)
    for word in word_list:
        word = word.lower()
        vocab.add(word)
        temp_vec[word] += 1
    return normalize_map(temp_vec)

def rocchio_training():
    with open("training_data.json") as data_file:
        for line in data_file:
            data = json.loads(line)
            label = data["label"]
            doc_count[label] += 1
            review = data["text"]
            doc_vec = tokenize(review)
            for word in doc_vec:
                klass_vec[label][word] += doc_vec[word]

def gen_centroid_vecs():
    for klass in klasses:
        num_docs = doc_count[klass]
        for word in vocab:
            klass_vec[klass][word] = float(klass_vec[klass][word]) / num_docs
            
def manhattan_dist(klass, doc_vec):
    dist = 0.0
    for word in vocab:
        dist += abs(klass_vec[klass][word] - doc_vec[word])
    return dist

def euclidean_dist(klass, doc_vec):
    dist = 0.0
    for word in vocab:
        dist += ((klass_vec[klass][word] - doc_vec[word])*(klass_vec[klass][word] - doc_vec[word]))
    
    return math.sqrt(dist)

def rocchio_classify_manhattan(review):
    doc_vec = tokenize(review)
    min_dist = sys.maxint
    predicted_klass = None
    for klass in klasses:
        '''
        dist = 0.0
        for word in vocab:
            dist += abs(klass_vec[klass][word] - doc_vec[word])
        '''
        #dist = manhattan_dist(klass, doc_vec)
        dist = euclidean_dist(klass, doc_vec)
        if dist < min_dist:
            min_dist = dist
            predicted_klass = klass
    return predicted_klass


rocchio_training()
gen_centroid_vecs()

In [54]:
# Apply your classifier on the test data. Report the results.
# Insert as many cells as you want
def test():
    incorrect_docs = {}
    total_docs = 0.0
    confusion_matrix = defaultdict(lambda: defaultdict(lambda: 0.0))
    with open("testing_data.json") as data_file:
        for line in data_file:
            total_docs += 1.0
            data = json.loads(line)
            review = data["text"]
            predicted_klass = rocchio_classify_manhattan(review)
            if predicted_klass != data["label"]:
                incorrect_docs[total_docs] = (data["text"], data["label"])
                confusion_matrix[predicted_klass]["False"] += 1
            else:
                confusion_matrix[predicted_klass]["True"] += 1
    
    print "accuracy : ", float(total_docs - len(incorrect_docs)) / total_docs
    print klasses[0], " : "
    print "Precision : ", float(confusion_matrix[klasses[0]]["True"]) / (confusion_matrix[klasses[0]]["True"] + confusion_matrix[klasses[0]]["False"])
    print "Recall : ", float(confusion_matrix[klasses[0]]["True"]) / (confusion_matrix[klasses[0]]["True"] + confusion_matrix[klasses[1]]["False"])
                                                                                       
    print klasses[1], " : "
    print "Precision : ", float(confusion_matrix[klasses[1]]["True"]) / (confusion_matrix[klasses[1]]["True"] + confusion_matrix[klasses[1]]["False"])
    print "Recall : ", float(confusion_matrix[klasses[1]]["True"]) / (confusion_matrix[klasses[1]]["True"] + confusion_matrix[klasses[0]]["False"])  
    '''
    #print len(incorrect_docs), " ", total_docs
    for key in incorrect_docs:
        #print key, " : ", incorrect_docs[key]
        print key
    '''
test()

accuracy :  0.67
Food-relevant  : 
Precision :  0.818181818182
Recall :  0.661764705882
Food-irrelevant  : 
Precision :  0.488888888889
Recall :  0.6875


# Part 3: Naive Bayes vs. Rocchio [20 points]

Which method gives the better results? In terms of what? How did you compare them? Can you explain why you observe what you do? Write 1-3 paragraphs below.

**Add your answer here:**

# Part 4: Recommenders [10 points]

Finally, since we've begun our discussion of recommenders, let's do a quick problem too:

The table below is a utility matrix, representing the ratings, on a 1–5 star scale, of eight items, *a* through *h*, by three users *A*, *B*, and *C*. 
<pre>

  | a  b  c  d  e  f  g  h
--|-----------------------
A | 4  5     5  1     3  2
B |    3  4  3  1  2  1
C | 2     1  3     4  5  3

</pre>

Compute the following from the data of this matrix.

(a) Treating the utility matrix as boolean, compute the Jaccard distance between each pair of users.

(b) Repeat Part (a), but use the cosine distance.

(c) Treat ratings of 3, 4, and 5 as 1 and 1, 2, and blank as 0. Compute the Jaccard distance between each pair of users.

(d) Repeat Part (c), but use the cosine distance.

(e) Normalize the matrix by subtracting from each nonblank entry the average
value for its user.

(f) Using the normalized matrix from Part (e), compute the cosine distance
between each pair of users.

(g) Which of the approaches above seems most reasonable to you? Give a one or two sentence argument supporting your choice.

**Add your answer here:**

(a)

(b)

(c)

(d)

(e)

(f)

(g)